Introducing the Hubbard Model
=========================

Prerequisites: OpenFermion and SciPy (which will pull in NumPy), e.g. install with pip:

`$ python -m pip install openfermion scipy`

Introduction
--------------

This notebook introduces the Hubbard Model$^1$ for use with the Variational Quantum Eigensolver (VQE) algorithm. The Hubbard Model (HM) has a range of uses, most notably in solid-state physics to explain phase transitions and other correlated behaviors. There are several variations of the HM, but in essence, the model describes interacting particles on a lattice. Despite its apparent simplicity, it is hard to solve on a classical computer because the known algorithms scale exponentially unless further approximations are made. In this tutorial, we will use the VQE algorithm to estimate the energy ground states of different configurations of a HM.

The Hubbard Model is useful for testing a quantum software stack, because it is easy to scale without changing its fundamental behavior, allowing apple to apple comparisons. Specific to VQE, its properties make it possible, within limits, to separately change the complexity of the quantum and the classical parts of the algorithm, stressing each individually. Furthermore, the resource requirements for running a full software chain on small configurations are low enough to fit and run on a laptop, within the time constraints of a software tutorial setting.

Constructing a Hubbard Model
------------------------------------

There are many variations on the Hubbard Model. Here we'll restrict ourselves to an HM with fermions (such as electrons), or the Fermi-Hubbard Model, as it is one of the simplest to understand while having enough parameters to modify resource usage for scaling of the quantum part of VQE, and for changing the (a)symmetry to control the complexity for the classical minimizer step in VQE.

### Basic Fermi-Hubbard Model

<p>
A visualization of an example of the Fermi-Hubbard Model, on a 2x2 grid with spinful electrons, is shown below. Electrons being fermions means that two electrons can not occupy the same quantum state. Because the electrons have spin, there can thus be at most two electrons per site (one spin-up, one spin-down). The model is characterized by an interaction term 'U', which only contributes if a spin-up and a spin-down electron occupy the same site; and by a "hopping" term 't', representing the kinetic energy in the system. In this basic model, only hops to neighboring sites are considered.

<table style="padding: 0px"><tr></tr><tr>
<td style="width: 65%; padding: 0px">
<div align="left" style="text-align: left; font-size: 120%">
The energy in the model is described by the following Hamiltonian:
<br>
<p>\begin{equation*}
H = -t \sum_{ij,\sigma=\uparrow\downarrow} {a_{i\sigma}^\dagger a_{j\sigma}} + U \sum_i {a_{i\uparrow}^\dagger a_{i\uparrow} a_{i\downarrow}^\dagger a_{i\downarrow}}
\end{equation*}</p>
where:
<ul>
    <li>$a_{i\sigma}^\dagger$ = creation operator for site 'i' and spin $\sigma$</li>
    <li>$a_{i\sigma}$ = annihilation operator for site 'i' and spin $\sigma$</li>
    <li>U = interaction term</li>
    <li>t = hopping term</li>
</ul>
<p>The first sum quantifies moving electrons from each site 'j' to each neighboring site 'i' (first term, that is: $a_i^\dagger$ "creates" an electron at site 'i' while $a_j$ simultaneously "annihilates" it from site 'j'), and vice versa (second term), considering both up and down spins ($\sigma$) independently; the second sum quantifies the interaction contribution for having two electrons, one up and one down, at the same site 'i'.
</p></div></td>
    <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
<td style="width: 35%">
    <img src='./figs/basic_fermi_hubbard_model.png'>
</td>
</tr></table>

<p>
Although the pictorial shows the electrons as concrete particles, the model treats them as waves that extend over all sites. A solution to the model provides the probabilities with which any of the electrons can be measured, as a particle, at each one of the sites.

### OpenFermion

The OpenFermion$^{2,3}$ package provides an implementation of the Fermi-Hubbard model in terms of `FermionOperators`$^4$, representing the $a_i$ and $a_i^\dagger$ terms. The Hamiltonian as shown above can then be transliterated directly by the appropriate operators and a multiplication coefficient.

`FermionOperators` conform to the following algebra:
<br>
<p><div style="text-align: left">
\begin{align}
\{a_i, a_j^\dagger\} & = \delta_{ij} \\
\{a_i^\dagger, a_j^\dagger\} = \{a_i, a_j\} & = 0 \\
|S\rangle & = \Pi_{j=0}^{N} (a_j^\dagger)^{k_j} |\oslash\rangle
\end{align}
</div></p>
where the first two anti-commutation relations (def. $\{A, B\} \equiv AB + BA$) ensure that no two electrons can occupy the same quantum state (the definition of being a fermion) and the last equation codifies the behavior of the creation operator (independently "creating" electrons from the vacuum state for each state $j$), meaning that an eigenstate for one electron at $j$ is simultaneously an eigenstate for all others, thus forming an orthogonal basis.

From these definitions, it follows that there exists a hermitian, and thus observable, operator $a_j^\dagger a_j$ with eigenvalues `0` and `1`. In effect, a counter of electrons (see later, below). It also follows that if $|\psi\rangle$ is an eigenstate of $a_j^\dagger a_j$ with eigenvalue `1` (i.e. an electron present), then $a_j |\psi\rangle$ is also an eigenstate of $a_j^\dagger a_j$, but with eigenvalue `0` (i.e. $a_j$ "annihilates" the electron). This is because $a_j^\dagger a_j a_j$ is `0`, because $a_j a_j$ is `0` (from the anti-commutation relations). Similarly, $a_j^\dagger$ can be shown to "create" an electron if the current state is an eigenstate with eigenvalue `0`, i.e. it yields the eigenstate with eigenvalue `1`. Applying either $a_j$ or $a_j^\dagger$ a second time yields `0` in both cases.

With that, we can now use OpenFermion to create a Hubbard Model representing a 2D grid of 2x2 for spinful electrons, as visualized above.

In [ ]:
import openfermion as of
import scipy.linalg as spla
import numpy as np

In [ ]:
x_dim    = 2
y_dim    = 2       # 2D 2x2 grid
t        = 1.0     # hopping term
U        = 2.0     # interaction term
spinless = False   # double negative, so do have spins

In [ ]:
hm_fermion = of.fermi_hubbard(
    x_dimension = x_dim,
    y_dimension = y_dim,      # 2D x_dim * y_dim grid
    tunneling   = t,          # hopping term
    coulomb     = U,          # interaction term (electric, hence 'coulomb')
    spinless    = spinless,   # double negative, so do have spins
    periodic    = False,      # do not allow wrap-around hopping
)
print(hm_fermion)

Printing the generated model shows its terms. Clearly visible are 'U' (`2.0`) and '-t' (`-1.0`) as multipliers. States are labeled from `0` to `7`, with two states per site (spin-up and spin-down; e.g. `0` to indicate $(x,y) = (0,0)$, spin-up; and `1` for $(x,y) = (0,0)$, spin-down, where order does not matter at this point because of symmetry). A caret (`^`) indicates a creation operator, thus a term such as '$-1.0\ [0^\wedge\ 2]$' stands for a hop of an electron from $(x,y) = (0,1)$ ("annihilated" from `2`) to $(x,y) = (0,0)$ ("created" at `0`). As can be seen, there are no hopping terms from even to odd or from odd to even states, thus the hopping terms _conserve spin_.

**Exercise:** Work through one of the interaction terms, such as for example '2.0 [0^ 0 1^ 1]', as was done above for a hopping term and convince yourself that it is indeed correct.

**Optional Exercise:** Change the input parameters to `openfermion.fermi_hubbard` and verify that the terms change as expected. For example, test what happens if `periodic = True` for a larger grid that has sites with different connectivity (e.g. 3x3); or set `spinless = True` to see the number of sites (and thus terms) drop accordingly.

In [ ]:
# Optional Exercise: build the Hubbard Model with various different parameters
# and verify their effect on the fermion terms. (Hint: rather than printing
# `hm_fermion_a`, which may give a lot of output, you can also access the terms
# directly, e.g. the total number: `len(hm_fermion_a.terms)`

x_dim_a    = 2       # number of sites in x dimension of grid
y_dim_a    = 2       # idem for y dimension
t_a        = 1.0     # hopping term
U_a        = 2.0     # interaction term
spinless_a = False   # double negative (`False` means "have spins")
periodic_a = False   # whether to allow wrap-around hopping (`False` means "no")

hm_fermion_a = of.fermi_hubbard(
    x_dimension = x_dim_a,
    y_dimension = y_dim_a,
    tunneling   = t_a,
    coulomb     = U_a,
    spinless    = spinless_a,
    periodic    = periodic_a,
)
print(hm_fermion_a)

### Extended Fermi-Hubbard Model

The values of `U` and `t` affect the optimization surface as seen by the classical optimizer in VQE. For example, by having the same sign (and thus terms with an opposite sign in the Hamiltonian) and `U` being a multiple of `t`, there will be more than one configuration for the ground level energy.

OpenFermion provides a few more extensions, in particular a chemical potential $\mu$ and a magnetic field $h$ can be specified. If non-zero, the following terms are added to the Fermi-HM Hamiltonian:
<p>\begin{equation*}
    - \mu \sum_i \sum_{\sigma} a^\dagger_{i\sigma} a_{i\sigma}
             - h \sum_i (a^\dagger_{i\uparrow} a_{i\uparrow} -
                       a^\dagger_{i\downarrow} a_{i\downarrow})
\end{equation*}</p>
Adding the chemical potential term doesn't change the optimization surface, but does increase the depth of the quantum circuit as generated. The magnetic field term creates an asymmetry between spin-up and spin-down, which translates into asymmetries in the optimization surface when there are fewer electrons of one or the other type.

**Optional Exercise:** Change the input parameters to `openfermion.fermi_hubbard`, adding the `chemical_potential` and  `magnetic_field` parameters, then verify that the terms change as expected from the added terms in the Hamiltonian.

In [ ]:
# Optional Exercise: build the Hubbard model with various different parameters
# for the chemical potential and the magnetic field; verify their effect on
# the fermion terms

chemical_potential_a = 0.0
magnetic_field_a     = 0.0

hm_fermion_a = of.fermi_hubbard(
    x_dimension        = x_dim_a,
    y_dimension        = y_dim_a,
    tunneling          = t_a,
    coulomb            = U_a,
    chemical_potential = chemical_potential_a,
    magnetic_field     = magnetic_field_a,
    spinless           = spinless_a,
    periodic           = periodic_a,
)
print(hm_fermion_a)

Mapping to qubits
---------------------

Fermion algebra is only a subset of what a quantum computer can express, since fermions are interchangeable and many states are thus completely equivalent, whereas qubits are identifiable (each one has a well defined location on a chip or ion trap, for example). Thus, it is useful to perform a mapping from fermion to qubit algebra that preserves the fermionic relations, to ensure that the mapped operators only produce states inside that fermionic subset; meaning that the computation does not involve non-physical states. Doing so reduces the scope for errors.$^5$

Whereas fermion algebra is expressed in terms of creation and annihilation operators, qubit algebra uses the familiar Pauli operators, i.e. the following set of 2x2 matrices and their corresponding commutation (def. $[A, B] \equiv AB - BA$) relations:

<table><tr></tr><tr>
<td>
<div>
$$
X = \left ( \begin{matrix}
0 & 1 \\
1 & 0
\end{matrix} \right )
\ \ \ 
Y = \left ( \begin{matrix}
0 & -i \\
i & 0
\end{matrix} \right )
\ \ \ 
Z = \left ( \begin{matrix}
1 & 0 \\
0 & -1
\end{matrix} \right )
\ \ \ 
I = \left ( \begin{matrix}
1 & 0 \\
0 & 1
\end{matrix} \right )
$$
</div>
</td><tr></tr>
<tr><td>
<div style="text-align: left">
\begin{align}
[X, Y] & = 2iZ \\
[Z, X] & = 2iY \\
[Y, Z] & = 2iX
\end{align}
</div>
</td></tr></table>

**Optional Exercise:** Verify the Pauli operators' commutator relations.

In [ ]:
# Optional Exercise: verify the Pauli operators' commutator relations
X = np.array([[0,  1   ], [1,     0]])
Y = np.array([[0,  0-1j], [0+1j,  0]])
Z = np.array([[1,  0   ], [0,    -1]])

assert (X.dot(Y)-Y.dot(X) == 2j*Z).all()
# etc.

There are several possible mappings from fermion to qubit operators that preserve the fermion algebra. Here, we'll consider the Jordan-Wigner$^6$ and Bravyi-Kitaev$^7$ transforms, because these are commonly used and easy to understand. In OpenFermion, Jordan-Wigner is the default choice almost across the board and the notebooks will stick with it in many places, because it is the simplest and therefore more convenient for educational use. However, certainly for our basic Fermi-Hubbard Model where many operations are nearest neighbor, the improved locality in the Bravyi-Kitaev transform yields significantly shorter circuits that in addition, are easier to optimize.$^8$

### Jordan-Wigner ###

The Jordan-Wigner$^6$ transform is defined by:
<p>\begin{align}
a_k^\dagger & = \frac{1}{2} (X_k - iY_k) Z_1 Z_2 Z_3 ... Z_{k-1} \\
a_k & = \frac{1}{2} (X_k + iY_k) Z_1 Z_2 Z_3 ... Z_{k-1}
\end{align}</p>
<br>A couple of things stand out immediately: first, the mapping is one-to-one (fermion index $k$ maps to qubit index $k$). That is, each fermionic quantum state (site + spin) is mapped onto a single qubit, representing the presence ($|1\rangle$) or absence ($|0\rangle$) of an electron (this is called the "occupation number basis"). Second, each operation involves a string of operations back onto all previously mapped states, to ensure the correct parity, and consequently the operations are non-local, with the last mapping (potentially) operating on <i>all</i> qubits. On a noisy quantum computer, this means that the last fermion operator will have an outsized impact on noise in the final result, which may lead to bias. And third, there are some clear inefficiencies: for example, two qubits are used to indicate that there is no electron (neither spin-up or spin-down) at a site. To this last point, however, a more complex encoding would involve more operations and at small scale there isn't any benefit to that trade-off.$^9$

<p>We can now calculate the necessary resources to simulate a chosen model, as each possible fermionic state requires 1 qubit. If the modeled electrons are chosen to have spins, this then requires twice the number of sites. If no spins (`spinless = True`), then the total is simply the number of sites itself.

<br>**Exercise:** Calculate the number of qubits needed under the Jordan-Wigner mapping for the `hm_fermion` model above.

In [ ]:
# Exercise: calculate required qubit resources
if spinless:
    n_qubits = ...
else:
    n_qubits = ...
print("Number of qubits required:", n_qubits)

In `QubitOperators`, the hopping and interaction terms under Jordan-Wigner become:
<br><p>\begin{align}
a_j^\dagger a_k + a_k^\dagger a_j & = \frac{1}{2} (X_jX_k + Y_jY_k) Z_{j+1}...Z_{k-1} \\
a_j^\dagger a_j a_k^\dagger a_k & = \frac{1}{2} (I - Z_j)(I - Z_k)
\end{align}</p>

Note that the hopping term groups together the move from one site to another (preserving the total number of electrons), as was also the case in the generated model, and that spin conservation is implied in the indices.

**Optional Exercise:** Verify that the terms above are correct, by constructing one component in terms of `of.FermionOperator`s and transforming it with `of.transforms.jordan_wigner`. Note that OpenFermion will optimize the $Z$ terms, e.g. removing any non-relevant global phase.

In [ ]:
# Optional Exercise: verify the results of a transformation for
# some choice of indices
f = of.FermionOperator
jw = of.transforms.jordan_wigner

# hopping term example
term = f('5^')*f('1') + f('1^')*f('5')
print(jw(term))

# interaction term example
term = ...

### Bravyi-Kitaev ###

The string of $Z$ gates over preceding qubits to ensure parity under the Jordan-Wigner transformation is a consequence of the direct one-to-one mapping of fermionic states to qubits. That is, in order to figure out the parity as part of an operator, you need to run over all other electrons that are potentially affected, and thus all qubits that are potentially affected. An obvious alternative, storing the parity of each state in a qubit (the "parity basis") as opposed to that state's occupancy, will make the latter commensurately difficult, and thus improve nothing.

As an alternative approach, the Bravyi-Kitaev$^7$ transformation maps the electron configuration to the string of qubits with an encoding that includes both occupancy and partial parity information (that of all sites up to the current index, if the index is odd). In effect, _both_ occupancy and parity information are then stored non-locally, as opposed to having occupancy local and parity non-local as is the case for Jordan-Wigner. The result is an overall more balanced approach.

Given some configuration electrons, up and down, over fermion sites $\vec{f}$, the mapping onto a qubit string $\vec{q}$ under Bravyi-Kitaev becomes:

<p>\begin{equation*}
    q_j = j \text{ mod } 2 \text{ ? } f_j + \sum_{i=0}^j f_i : f_i
\end{equation*}</p>

where the sum is also mod $2$. That is, the qubit stores occupancy if the index is even, and _partial_ parity information combined with occupancy if odd.

The Bravyi-Kitaev transform applied to the creation and annihilation operators then results in:
<p>\begin{align}
a_k^\dagger & = \frac{1}{2} (X_kZ_{k-1} - iY_k) X_{k+1}X_{k+2} ... X_N \\
a_k & = \frac{1}{2} (X_kZ_{k-1} + iY_k) X_{k+1}X_{k+2} ... X_N
\end{align}</p>

***

### Intermezzo: Labeling and Connectivity ###

Not all combinations of $j$ and $k$ indices appear in our model: hopping is only allowed between neighboring sites on the grid and is spin-preserving; interaction only occurs between opposite spins at a site. Since fermions are interchangeable, we're free to choose any qubit labeling from state to qubit and if the quantum computer does not have all-to-all connectivity, labeling will make a difference for the overall circuit depth. (Note that the labeling is a practical, not mathematical, matter: the Jordan-Wigner and Bravyi-Kitaev transforms are fully reversible.) When OpenFermion performs the either transform on a FermionOperator, it will leave the existing index ordering in place, thus the labeling can be controlled by changing the indices in the fermion terms.

Some examples of common label orderings are:

<table><tr></tr><tr>
<td style="width: 33%; text-align: center; font-size: 120%">
    <p><img src='./figs/no_spin_matrix_order.png'></p>
    <br><p><center>matrix ordering, no spin</center></p>
</td>
<td>&nbsp;</td>
<td style="width: 33%; text-align: center; font-size: 120%">
    <p><img src='./figs/no_spin_snake_order.png'></p>
    <br><p><center>snake ordering, no spin</center></p>
</td>
<td>&nbsp;</td>
<td style="width: 33%; text-align: center; font-size: 120%">
    <p><img src='./figs/spin_snake_order.png'></p>
    <br><p><center>snake ordering, with spin</center></p>
</td>
</tr></table>

Note that the last ("snake") ordering effectively captures the fact that fermions of different spin can change state independently (i.e. the overall Hilbert space can be split in two sub-spaces, which isn't true for subsets of electrons of the same spin).

After the Jordan-Wigner transformation, having chosen a labeling, adding electrons to model simulation is a simple matter of applying an X-gate to the appropriate qubits to change their state from $|0\rangle$ to $|1\rangle$, thus "filling" them. After the Bravyi-Kitaev transformation, given the labeling, electrons are added by applying an X-gate to the qubit representing the occupancy (even indices) or the sum of occupancy and partial parity (odd indices).

***

Solving the Hubbard Model Classically
---------------------------------------------

An analytical solution is only known for the 1D Hubbard Model.$^{10}$ Another approach is exact diagonalization of the operator matrix. Unfortunately, the matrix size grows exponentially with the number of states, so that quickly becomes infeasible (at least, for classical computations). However, since in this tutorial we're only considering relatively small models (to be able to classically simulate them as run on a quantum computer, a process that is likewise size-limited), the exact diagonalization will do fine to solve our model.

The exact diagonalization will provide the eigenvalues and eigenvectors of the model. However, the overall energy in the system depends on the number of electrons we added to the model. Given a state, represented by an eigenvector, we thus also need to know for how many electrons (and whether up or down) it occurs. Since the overall number of electrons as well as their spin numbers are conserved, we're only interested in states that represent a whole number of electrons.

Thus, we need some operators to count the electrons, given an input state. They are constructed in the following code block and consist of applying the, earlier introduced, counter operator $a_j^\dagger a_j$ to the relevant sites.

In [ ]:
# Fermion operators for counting the number of spin-up and spin-down electrons
n_qubits = x_dim * y_dim * (spinless and 1 or 2)
spin_up_fermion   = sum([of.FermionOperator(((i, 1), (i, 0))) for i in range(0, n_qubits, 2)])
spin_down_fermion = sum([of.FermionOperator(((i, 1), (i, 0))) for i in range(1, n_qubits, 2)])

**Exercise:** Print the `spin_up` and `spin_down` operators to verify that their terms are as expected. Construct an operator to count the total number of electrons, regardless of spin.

In [ ]:
# Exercise: print the `spin_up` and `spin_down` operators and verify
# their terms (hint: compare them with the interaction term in the
# original Hamiltonian)
print('spin-up operator:\n%s' % spin_up_fermion)
print('spin-down-operator:\n%s' % spin_down_fermion)

# Exercise: construct an operator to count the total number of electrons.
# (Hint: think about what the terms would look like and how the fermion
# operators then fit together.)
number_op = ...
print('number operator:\n%s' % number_op)

Now we're ready to calculate the lowest possible energy levels of the model and the configurations to which they correspond. First, all operators are converted to matrices to work with SciPy's `eigh`:

In [ ]:
# Note: make sure express all operators in the same encoding; keep
# them as FermionOperators or use the same mapping to qubits for all

# Note: the up_matrix is of smaller dimensions because its operator only
# runs through 7 qubit indices, so add identity (no-op) for the 8th
hm_matrix   = of.get_sparse_operator(hm_fermion).todense()
up_matrix   = np.kron(of.get_sparse_operator(spin_up_fermion).todense(), np.eye(2))
down_matrix = of.get_sparse_operator(spin_down_fermion).todense()

print('Hubbard Model operator matrix dimensions:', hm_matrix.shape)

Next, calculate the expectation values of the operators and select only the ones with whole numbers of electrons (up and down). Keep the ones with the lowest energy levels for each configuration and print them.

In [ ]:
eigenvalues, eigenvectors = spla.eigh(hm_matrix)

energies = {}
for i in range(hm_matrix.shape[0]):
    v = eigenvectors[:,i]
    n_up = float(np.real(v.T.dot(up_matrix).dot(v)))
    if not round(n_up, 3).is_integer():
        continue
    
    n_down = float(np.real(v.T.dot(down_matrix).dot(v)))
    if not round(n_down, 3).is_integer():
        continue

    n_up, n_down = round(n_up), round(n_down)
    level = np.real(v.T.dot(hm_matrix).dot(v))
    energies[n_up, n_down] = min(energies.get((n_up, n_down), np.inf), level)

print('energy levels:')
for (n_up, n_down), level in energies.items():
    print('%d up and %d down : % .5f' % (n_up, n_down, level))

**Optional Exercise:** modify the code above to print the energy levels of all physical configurations. What does it mean if there are multiple energy levels (eigenvalues) for the same configuration?

In [ ]:
# Optional Exercise: modify the code above (repeated below) to print the
# energy levels of all physical configurations
energies = {}
for i in range(hm_matrix.shape[0]):
    v = eigenvectors[:,i]
    n_up = float(np.real(v.T.dot(up_matrix).dot(v)))
    if not round(n_up, 3).is_integer():
        continue
    
    n_down = float(np.real(v.T.dot(down_matrix).dot(v)))
    if not round(n_down, 3).is_integer():
        continue

    n_up, n_down = round(n_up), round(n_down)
    level = np.real(v.T.dot(hm_matrix).dot(v))
    energies[n_up, n_down] = min(energies.get((n_up, n_down), np.inf), level)

print('energy levels:')
for (n_up, n_down), level in energies.items():
    print('%d up and %d down : % .5f' % (n_up, n_down, level))

References
-------------

[1] J. Hubbard, Proc. Roy. Soc. (Lond.) 276 (1963) 238
<br>[2] https://arxiv.org/abs/1710.07629, McClean et al., "OpenFermion: The Electronic Structure Package for Quantum Computers"
<br>[3] https://github.com/quantumlib/OpenFermion
<br>[4] https://quantumai.google/reference/python/openfermion/ops/FermionOperator
<br>[5] https://arxiv.org/abs/cond-mat/0012334, Ortiz et al., "Quantum Algorithms for Fermionic Simulations"
<br>[6] P. Jordan and E. Wigner, "Über das Paulische Äquivalenzverbot", Z. Phys. 47, 631 (1928)
<br>[7] https://arxiv.org/abs/quant-ph/0003137 S. Bravyi, A. Kitaev, "Fermionic quantum computation"
<br>[8] https://arxiv.org/abs/1812.02233, A. Tranter, et al., "A comparison of the Bravyi-Kitaev and Jordan-Wigner transformations for the quantum simulation of quantum chemistry"
<br>[9] https://arxiv.org/abs/1912.06007, Cade et al., "Strategies for solving the Fermi-Hubbard model on near-term quantum computers"
<br>[10] E.H. Lieb, F.Y. Wu, "Absence of Mott transition in an exact solution of the short-range, one-band model in one dimension", Phys. Rev. Lett. 20 (1968) 1445–1448.